# Pre-Cruise Data Aggregator

Aggregates publicly available oceanographic datasets for a geographic region of interest (AOI) into a single GeoPackage (`.gpkg`) file. The output file can be dragged and dropped directly into QGIS or ArcGIS — all layers will appear with their names and correct projection.

### Currently supported datasets:
- **GMRT Bathymetry** — Masked and unmasked seafloor bathymetry grids
- **Cruise Tracks** — Historical research vessel tracks from the GMRT archive

### Planned:
- GBIF (Biological Occurrences)
- OBIS (eDNA & Occurrences)
- IMLGS (Geological Samples)
- GLODAP (Water Chemistry)
- WCSD (Water Column Sonar Footprints)

### How to use this notebook:
1. Edit the **bounding box coordinates** in the "Define Area of Interest" cell below
2. **Run All Cells** (Cell menu > Run All, or the run-all button in your toolbar)
3. Find the output GeoPackage in the `data/` folder

In [ ]:
import os
import sys
import requests
import wget
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box, Polygon
import rasterio
from pygbif import occurrences as gbif_occ
from pyobis import occurrences as obis_occ
import json
import io
import zipfile

DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

## Define Area of Interest (AOI)

**This is the only cell you need to edit.** Set the bounding box coordinates for your cruise region below. Coordinates are in decimal degrees (WGS 84).

The example below covers the Blake Plateau / Southeast US continental margin.

In [ ]:
# --- EDIT THESE COORDINATES ---
# EXAMPLE: BLAKE PLATEAU / SOUTHEAST US
MIN_LAT = 28.0   # Southern boundary
MAX_LAT = 32.0   # Northern boundary
MIN_LON = -80.0   # Western boundary
MAX_LON = -76.0   # Eastern boundary
# ------------------------------

# TODO test all combinations of +/-180, 360 lat/long

# Build a rectangular polygon from the bounding box for clipping data later
aoi_polygon = box(MIN_LON, MIN_LAT, MAX_LON, MAX_LAT)
aoi_wkt = aoi_polygon.wkt

print(f"Area of Interest defined: {aoi_wkt}")

# All downloaded layers will be assembled into this GeoPackage
OUTPUT_FILENAME = os.path.join(DATA_DIR, "PreCruise_Data_Package.gpkg")

## Get Bathymetry

Downloads seafloor bathymetry from the [GMRT synthesis](https://www.gmrt.org) using the GridServer API. Two versions are retrieved:

- **Unmasked** — includes interpolated bathymetry in areas without direct survey coverage
- **Masked** — only includes areas with actual multibeam sonar data

You can adjust the `RESOLUTION` setting below. Options are `low`, `default`, `med`, `high`, and `max`. Higher resolutions produce larger files — very large requests may time out or fail.

API documentation: https://www.gmrt.org/services/gridserverinfo.php#!/services/getGMRTGrid

In [ ]:

# Create bathymetry landing directory
BATHY_DIR = os.path.join(DATA_DIR, "bathymetry")
os.makedirs(BATHY_DIR, exist_ok=True)

# GMRT resolution entries can be low/default, med, high, max
RESOLUTION = 'default'

# GMRT file types can vary, but recommend GeoTiff or Coards NetCDF grid
format = 'coards'  # geotiff, coards

# Create url for GMRT API call from coordinates - unmasked
gmrt_url_unmasked = (f'https://www.gmrt.org/services/GridServer?north={MAX_LAT}'
                     f'&west={MIN_LON}&east={MAX_LON}&south={MIN_LAT}'
                     f'&layer=topo&format={format}&resolution={RESOLUTION}')

# Create url for GMRT API call from coordinates - unmasked metadata
gmrt_url_unmasked_metadata = (f'https://www.gmrt.org/services/GridServer/metadata?north={MAX_LAT}'
                              f'&west={MIN_LON}&east={MAX_LON}&south={MIN_LAT}'
                              f'&format={format}&mformat=json&resolution={RESOLUTION}')

# Create url for GMRT API call from coordinates - masked
gmrt_url_masked = (f'https://www.gmrt.org/services/GridServer?north={MAX_LAT}'
                   f'&west={MIN_LON}&east={MAX_LON}&south={MIN_LAT}'
                   f'&layer=topo-mask&format={format}&resolution={RESOLUTION}')

# Create url for GMRT API call from coordinates - masked metadata
gmrt_url_masked_metadata = (f'https://www.gmrt.org/services/GridServer/metadata?north={MAX_LAT}'
                              f'&west={MIN_LON}&east={MAX_LON}&south={MIN_LAT}'
                              f'&format=geotiff&m{format}=json&resolution={RESOLUTION}')

print(f"Downloading bathymetry from\n{gmrt_url_unmasked}\n{gmrt_url_masked}\nto bathymetry folder")

# Use requests library to get the data
bathy_umask = requests.get(gmrt_url_unmasked, timeout=10)
bathy_mask = requests.get(gmrt_url_masked, timeout=10)
bathy_umask_metadata = requests.get(gmrt_url_unmasked_metadata, timeout=10)
bathy_mask_metadata = requests.get(gmrt_url_masked_metadata, timeout=10)

# set file extension
if format == 'coards':
    extension = 'grd'
elif format == 'geotiff':
    extension = 'tiff'
else:
    print("Invalid file extension")

# Check for success and write out file
if bathy_umask.status_code == 200:
    print("Retrieved umasked bathymetry")
    
    # parse the metadata for the file to a Python dictionary
    umask_metadata = bathy_umask_metadata.json()

    # construct filename and save the file
    bathy_unmasked_filename = f"{BATHY_DIR}/bathymetry_unmasked_{float(umask_metadata['meters_per_node']):.0f}m.{extension}"
    with open(bathy_unmasked_filename, "wb") as f:
        f.write(bathy_umask.content)
else:
    print(f"Error getting unmasked bathymetry: {bathy_umask.status_code}")

# Check for success and write out file
if bathy_mask.status_code == 200:
    print("Retrieved masked bathymetry")
    
    # parse the metadata for the file to a Python dictionary
    mask_metadata = bathy_umask_metadata.json()

    # construct filename and save the file
    bathy_masked_filename = f"{BATHY_DIR}/bathymetry_masked_{float(mask_metadata['meters_per_node']):.0f}m.{extension}"
    with open(bathy_masked_filename, "wb") as f:
        f.write(bathy_mask.content)

else:
   print(f"Error getting masked bathymetry: {bathy_mask.status_code}")


## Cruise Tracks

Downloads the full global cruise track shapefile from GMRT and clips it to the AOI. This includes historical research vessel tracks from surveys that contributed bathymetry to the GMRT synthesis.

In [ ]:

track_url = "https://www.gmrt.org/shapefiles/gmrt_cruise_tracks.zip"
filename = f"{DATA_DIR}/gmrt_cruise_tracks.zip"

# Check for and delete old cruise tracks file if found
if os.path.exists(filename):
    os.remove(filename)
    print(f"Removed old {filename}")

wget.download(track_url, out=filename)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f"{DATA_DIR}/cruise_tracks")

# trim cruise track shapefile to bounding coordinates
gdf_cruise_tracks = gpd.read_file(f"{DATA_DIR}/cruise_tracks/gmrt_cruise_tracks.shp")

trimmed_gdf = gdf_cruise_tracks.clip(aoi_polygon)
trimmed_gdf.to_file(f"{DATA_DIR}/cruise_tracks/trimmed_cruise_tracks.shp")

# Clean up global cruise tracks zip
os.remove(filename)
print(f"Removed {filename}")


## GBIF Biological Observations

*Not yet implemented.* Will download biological occurrence records from the [Global Biodiversity Information Facility](https://www.gbif.org/) within the AOI.

In [ ]:

# TODO query GBIF API with AOI polygon


## Assemble into GeoPackage

Combines all downloaded datasets into a single `.gpkg` file. This is the final output — drag and drop it into QGIS to load all layers at once.

In [ ]:
# Unmasked bathymetry - creates the GeoPackage
with rasterio.open(bathy_unmasked_filename) as src:
    unmasked_raster_data = src.read().astype(np.float32)
    kwargs = src.meta.copy()

    kwargs.update({
        'driver': 'GPKG',
        'count': src.count,
        'dtype': 'float32',
        'nodata': -99999.0,
        'crs': 'EPSG:4326'
    })

    with rasterio.open(OUTPUT_FILENAME, "w", RASTER_TABLE="bathymetry_unmasked", **kwargs) as dst:
        dst.write(unmasked_raster_data)

# Masked bathymetry - appends to existing GeoPackage
with rasterio.open(bathy_masked_filename) as src:
    masked_raster_data = src.read().astype(np.float32)
    kwargs = src.meta.copy()

    kwargs.update({
        'driver': 'GPKG',
        'count': src.count,
        'dtype': 'float32',
        'nodata': -99999.0,
        'crs': 'EPSG:4326'
    })

    with rasterio.open(OUTPUT_FILENAME, "w", RASTER_TABLE="bathymetry_masked", APPEND_SUBDATASET="YES", **kwargs) as dst:
        dst.write(masked_raster_data)

# Load trimmed cruise track shapefiles into GeoPackage
trimmed_gdf = gpd.read_file(f"{DATA_DIR}/cruise_tracks/trimmed_cruise_tracks.shp")
trimmed_gdf.to_file(OUTPUT_FILENAME, layer="cruise_tracks", driver="GPKG", mode="a")
print(f"Wrote layers to {OUTPUT_FILENAME}")


## Attributions

*Not yet implemented.* Will generate a citation/attribution report for the datasets included in the GeoPackage, using the GMRT attribution API and equivalent services for other data sources.

In [ ]:

# TODO bathymetry attributions XML parsing

# Create url for GMRT API call from coordinates - attributions
gmrt_url_attrib = (f'https://www.gmrt.org/services/GridServer/attribution?north={MAX_LAT}'
                   f'&west={MIN_LON}&east={MAX_LON}&south={MIN_LAT}'
                   f'&layer=topo&format=geotiff&resolution={RESOLUTION}')


# Get bathymetry attributions and write to text file
#report_path = os.path.join(BATHY_DIR, "data_attribution.txt")

#bathy_attrib = requests.get(gmrt_url_attrib, timeout=10)

